Exercise 2.5 (programming) Design and conduct an experiment to demonstrate the </br>
difficulties that sample-average methods have for nonstationary problems. Use a modified </br>
version of the 10-armed testbed in which all the q⇤(a) start out equal and then take</br>
independent random walks (say by adding a normally distributed increment with mean </br>
zero and standard deviation 0.01 to all the q⇤(a) on each step). Prepare plots like</br>
Figure 2.2 for an action-value method using sample averages, incrementally computed,</br>
and another action-value method using a constant step-size parameter, a = 0.1. Use </br>
eps = 0.1 and longer runs, say of 10,000 steps.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# initialize rewards
def assign_mean_rewards(n_actions, mean, variance):
    return np.random.normal(mean, variance, size=n_actions)

def assign_equal_rewards(n_actions):
    return np.zeros(n_actions)

In [3]:
# calculate reward
def get_reward(true_rewards, action, variance):
    reward_mean = true_rewards[action]
    return np.random.normal(reward_mean, variance)

def get_drifting_reward(true_rewards, action):
    """Drift all the rewards"""
    n = len(true_rewards)
    drifted_rewards = true_rewards + np.random.normal(0, 0.01, size=n)
    return drifted_rewards, true_rewards[action]

In [4]:
# policies
def greedy(qtable):
    return np.argmax(qtable)

def epsilon_greedy(qtable, n_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, n_actions)
    else:
        return np.argmax(qtable)

def ucb(qtable:np.ndarray, counts:np.ndarray, timestamp:int, exploration_rate:float):
    # argmax(qtable + c * ((ln(timestamp) / n)**0.5) )
    # first try all actions once
    zero_mask = (counts == 0)
    if zero_mask.any():
        # pick the first untried action
        a = int(np.flatnonzero(zero_mask)[0])
        counts[a] += 1
        return a, counts
    
    uncertainty = (np.log(timestamp+1) / counts)**0.5
    chosen_action  = int(
        np.argmax(qtable + exploration_rate * uncertainty)
    )
    counts[chosen_action] += 1

    return chosen_action, counts

def gradient_bandit(preferences:np.ndarray):
    """Calculates probabilities of all actions based on their `preference` value"""
    probabilities = []
    for action in (range(len(preferences))):
        prob = np.exp(preferences[action]) / np.sum(np.exp(preferences))
        probabilities.append(prob)
    return probabilities

In [5]:
# qtable updates
def update_estimate_average(qtable, true_reward, action, timestamp):
    current_estimate = qtable[action]
    # update our reward estimate by sample averaging
    updated_estimate = current_estimate + (true_reward - current_estimate) / (timestamp + 1)
    qtable[action] = updated_estimate
    return None

def update_estimate_alpha(qtable, true_reward, action, alpha):
    current_estimate = qtable[action]
    # update our reward estimate by alpha
    updated_estimate = current_estimate + alpha * (true_reward - current_estimate)
    qtable[action] = updated_estimate
    return None

def update_unbiased_constant(qtable, true_reward, action, timestamp, alpha):
    # formula to calculate trace at timestamp t: `1 + (1-alpha)^t  * (trace_0 - 1)`
    # where trace_0 is an initial constant trace value. In the textbook, it is set as 0 
    # which leaves us with the below formula:
    # 1 + (1-alpha) * (0-1) -> 1 + (1-alpha)^t * (-1) -> 1 - (1-alpha)^t
    trace = 1 - (1-alpha)**timestamp
    step_size = alpha / trace

    current_estimate = qtable[action]
    updated_estimate = current_estimate + step_size * (true_reward - current_estimate)
    qtable[action] = updated_estimate
    return None

def update_gradient(preferences, probabilities, true_reward, action, alpha, reward_baseline):
    # update preference for chosen action
    for a in range(len(preferences)):
        current_preference = preferences[a]
        if a == action:
            preferences[action] = current_preference + alpha * (true_reward - reward_baseline) * (1 - probabilities[a])
        else:
            preferences[action] = current_preference - alpha * (true_reward - reward_baseline) * probabilities[a]

def update_reward_baseline(baseline, new_reward, drifting, step_size):
    if drifting:
        return baseline + step_size * (new_reward - baseline)
    else:
        return baseline + (new_reward - baseline) / (step_size + 1)

In [6]:
# plotting
def plot_run_results(rewards):
    plt.figure(figsize=(12, 7))
    plt.plot(
        np.arange(len(rewards)), rewards
    )
    plt.title("Rewards over time")
    plt.show()

def plot_many_run_results(title:str, **kwargs):
    plt.figure(figsize=(12, 7))
    
    first_col = list(kwargs.keys())[0]
    x_axis = np.arange(len(kwargs[first_col])) 
    
    for name, values in kwargs.items():
        plt.plot(
            x_axis, values, label=name
        )
    
    plt.title(title)
    plt.legend()
    plt.show()

def plot_optimization_results(title:str, **kwargs):
    plt.figure(figsize=(12, 7))
    
    policies = list(kwargs.keys())
    
    for policy in policies:
        x_axis = list(kwargs[policy].keys())
        values = list(kwargs[policy].values())
        plt.plot(
            x_axis, values, label=policy
        )
    
    plt.title(title)
    plt.legend()
    plt.show()

In [7]:
def experiment(
        narms = 10,
        nruns = 2_000,
        eps = 0.01,
        alpha = 0.1,
        timestamps = 10_000,
        initial_reward = 0,
        policy='epsilon', # ucb, epsilon, greedy
        exploration_rate:int=2,
        drifting=True,
        variance=1,

        ):
    print(
        f"Running optimization for policy {policy} with parameters: {locals()}"
    )
    total_rewards = []
    total_choices = []

    if policy == 'ucb':
        counts = np.zeros(narms)
    elif policy=='gradient':
        reward_baseline = 0

    for n in range(nruns):
        # actual mean rewards for each action
        rewards_table = assign_equal_rewards(narms)
        
        # agent's estimate
        qtable = np.zeros(narms) + initial_reward # optimistic initialization
        bandit_rewards = []
        optimal_action = []

        for i in range(timestamps):
            # policies here
            if policy == 'epsilon':
                chosen_action = epsilon_greedy(qtable, narms, eps)
            elif policy == 'ucb':
                chosen_action, counts = ucb(qtable, counts, i, exploration_rate)
            elif policy == 'gradient':
                probabilities = gradient_bandit(qtable)
                chosen_action = np.argmax(probabilities)
            elif policy == 'greedy':
                chosen_action = greedy(qtable)
            
            if drifting:
                rewards_table, reward = get_drifting_reward(rewards_table, chosen_action)
            else:
                rewards_table, reward = get_reward(rewards_table, chosen_action, variance)

            if policy == 'gradient':
                update_gradient(qtable, probabilities, reward, chosen_action, alpha, reward_baseline)
                # update reward baseline for non-stationary problem
                step_size = alpha if drifting else i
                reward_baseline = update_reward_baseline(reward_baseline, reward, drifting, step_size)
            else:
                update_estimate_alpha(qtable, reward, chosen_action, alpha)

            optimal_action.append(
                int(chosen_action == np.argmax(rewards_table))
            )
            bandit_rewards.append(reward)
        
        total_rewards.append(bandit_rewards)
        total_choices.append(optimal_action)
        if n % (nruns // 10)==0 and n > 0:
            mean_reward = np.mean(total_rewards)
            print(
                f"Mean reward: {mean_reward} "
            ) 
    return total_rewards, total_choices

In [8]:
narms = 10
nruns = 2_000
alpha = 0.1 # only changes for gradient bandit 
timestamps = 10_000

In [9]:
hyper_param_list = [2 ** x for x in range(-7, 3)]

mean_rewards = {}
mean_optimal_choices = {}

In [10]:
policy = 'epsilon'
mean_rewards[policy] = {}
mean_optimal_choices[policy] = {}

for eps in hyper_param_list:
    if eps >= 1:
        break
    total_rewards, total_choices = experiment(
        narms=narms,
        nruns=nruns,
        eps=eps, # <-- hyper-parameter
        alpha=alpha,
        timestamps=timestamps,
        policy=policy,

    )
    # only consider the second half of timestamps for reward calculation
    mean_reward = np.mean(total_rewards[timestamps//2:]) 
    optimal_choice = np.mean(total_choices[timestamps//2:], axis=0) * 100.0

    mean_rewards[policy][eps] = mean_reward
    mean_optimal_choices[policy][eps] = mean_reward

# greedy
policy = 'greedy'
mean_rewards[policy] = {}
mean_optimal_choices[policy] = {}

for initial_reward in hyper_param_list:
    if initial_reward < 0.125:
        continue
    total_rewards, total_choices = experiment(
        narms=narms,
        nruns=nruns,
        eps=None,
        alpha=alpha,
        timestamps=timestamps,
        policy=policy,
        initial_reward=initial_reward,
        
    )

    # only consider the second half of timestamps for reward calculation
    mean_reward = np.mean(total_rewards[timestamps//2:]) 
    optimal_choice = np.mean(total_choices[timestamps//2:], axis=0) * 100.0

    mean_rewards[policy][initial_reward] = mean_reward
    mean_optimal_choices[policy][initial_reward] = mean_reward


# ucb
policy = 'ucb'
mean_rewards[policy] = {}
mean_optimal_choices[policy] = {}
for exploration_rate in hyper_param_list:
    if exploration_rate < 0.25:
        continue
    total_rewards, total_choices = experiment(
        narms=narms,
        nruns=nruns,
        eps=None,
        alpha=alpha,
        timestamps=timestamps,
        policy=policy,
        exploration_rate=exploration_rate,
        
    )

    # only consider the second half of timestamps for reward calculation
    mean_reward = np.mean(total_rewards[timestamps//2:]) 
    optimal_choice = np.mean(total_choices[timestamps//2:], axis=0) * 100.0

    mean_rewards[policy][exploration_rate] = mean_reward
    mean_optimal_choices[policy][exploration_rate] = mean_reward


# gradient
policy = 'gradient'
mean_rewards[policy] = {}
mean_optimal_choices[policy] = {}
for alpha in hyper_param_list:

    total_rewards, total_choices = experiment(
        narms=narms,
        nruns=nruns,
        eps=None,
        alpha=alpha,
        timestamps=timestamps,
        policy=policy,
        initial_reward=1, # just to avoid a 0 

    )

    # only consider the second half of timestamps for reward calculation
    mean_reward = np.mean(total_rewards[timestamps//2:]) 
    optimal_choice = np.mean(total_choices[timestamps//2:], axis=0) * 100.0

    mean_rewards[policy][alpha] = mean_reward
    mean_optimal_choices[policy][alpha] = mean_reward


Running optimization for policy epsilon with parameters: {'narms': 10, 'nruns': 2000, 'eps': 0.0078125, 'alpha': 0.1, 'timestamps': 10000, 'initial_reward': 0, 'policy': 'epsilon', 'exploration_rate': 2, 'drifting': True, 'variance': 1}
Mean reward: 0.8111447640830569 
Mean reward: 0.8153868032260645 
Mean reward: 0.8177093442649827 
Mean reward: 0.8271808637877979 
Mean reward: 0.8279553208514141 
Mean reward: 0.8295478703577016 
Mean reward: 0.8333767454746059 
Mean reward: 0.8300985925715351 
Mean reward: 0.8334377495421958 


c:\Users\pc\miniconda3\envs\rlagent\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\pc\miniconda3\envs\rlagent\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Running optimization for policy epsilon with parameters: {'narms': 10, 'nruns': 2000, 'eps': 0.015625, 'alpha': 0.1, 'timestamps': 10000, 'initial_reward': 0, 'policy': 'epsilon', 'exploration_rate': 2, 'drifting': True, 'variance': 1}
Mean reward: 0.9057598276194795 
Mean reward: 0.8977333217102692 
Mean reward: 0.899168649403954 
Mean reward: 0.8944048932171763 
Mean reward: 0.8863325178635166 
Mean reward: 0.8830150876149863 
Mean reward: 0.8786411823196456 
Mean reward: 0.8767350164538087 
Mean reward: 0.8730813763885346 
Running optimization for policy epsilon with parameters: {'narms': 10, 'nruns': 2000, 'eps': 0.03125, 'alpha': 0.1, 'timestamps': 10000, 'initial_reward': 0, 'policy': 'epsilon', 'exploration_rate': 2, 'drifting': True, 'variance': 1}
Mean reward: 0.9312917700120279 
Mean reward: 0.9210762946258086 
Mean reward: 0.915920738028263 
Mean reward: 0.922708289973953 
Mean reward: 0.919008394926651 
Mean reward: 0.916245808450364 
Mean reward: 0.9171591964699713 
Mean r

KeyboardInterrupt: 

In [ ]:
plot_optimization_results(
    title='optimization comparison',
    **mean_rewards
)